In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import xlrd 
import json


# # Import API key
# from api_keys import api_key


In [2]:
#import data from url to get the state abbr
url="http://worldpopulationreview.com/static/states/name-abbr.json"
d = requests.get(url).json()

states_abbr = pd.DataFrame(d.items() , columns=["State","State Abbr"])

states_abbr.head()

,State,State Abbr
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [3]:
#import excel file with obesity, unemployment, education
xls = pd.ExcelFile('Resources/2018 County Health Rankings Data - v2.xls')
df = pd.read_excel(xls, 'Ranked Measure Data', skiprows=1)


In [4]:
df.head()


,FIPS,State,County,Years of Potential Life Lost Rate,95% CI - Low,95% CI - High,Quartile,Years of Potential Life Lost Rate (Black),Years of Potential Life Lost Rate (Hispanic),Years of Potential Life Lost Rate (White),...,95% CI - High.20,Quartile.33,% Drive Alone (Black),% Drive Alone (Hispanic),% Drive Alone (White),# Workers who Drive Alone,% Long Commute - Drives Alone,95% CI - Low.21,95% CI - High.21,Quartile.34
0,1001.0,Alabama,Autauga,9409.294766,8492.111635,10326.477898,2,11806.223285,NaN,9148.381743,...,88.431482,4,82.494851,NaN,82.539683,21082.0,40.8,36.698437,44.901563,3
1,1003.0,Alabama,Baldwin,7467.596664,7025.900408,7909.292920,1,10335.702691,2827.348216,7484.117065,...,86.017804,2,85.567010,68.864800,84.646255,73058.0,40.1,37.614952,42.585048,3
2,1005.0,Alabama,Barbour,8929.474982,7633.929691,10225.020273,1,9141.333085,NaN,8896.000532,...,85.796041,1,80.341227,NaN,85.656228,7271.0,35.1,29.740946,40.459054,2
3,1007.0,Alabama,Bibb,11741.938889,10057.801626,13426.076153,4,11297.166352,NaN,11923.067966,...,89.905820,2,NaN,NaN,NaN,7044.0,48.8,40.894758,56.705242,4
4,1009.0,Alabama,Blount,9359.122343,8463.490290,10254.754396,1,NaN,5133.144570,9600.725685,...,87.622499,3,NaN,79.539642,81.580997,18384.0,60.2,55.818947,64.581053,4


In [5]:
#rename columns used for analysis
df.rename(columns={ df.columns[114]: "Unemployment Percent" }, inplace = True)
df.rename(columns={ df.columns[34]: "Obesity Percent" }, inplace = True)
df.rename(columns={ df.columns[108]: "Some College Percent" }, inplace = True)
df.rename(columns={ df.columns[104]: "High School Grad Percent" }, inplace = True)


#merge data
stats_df = pd.merge(states_abbr, df,
                                 how='left', on='State')

stats_df.head()


,State,State Abbr,FIPS,County,Years of Potential Life Lost Rate,95% CI - Low,95% CI - High,Quartile,Years of Potential Life Lost Rate (Black),Years of Potential Life Lost Rate (Hispanic),...,95% CI - High.20,Quartile.33,% Drive Alone (Black),% Drive Alone (Hispanic),% Drive Alone (White),# Workers who Drive Alone,% Long Commute - Drives Alone,95% CI - Low.21,95% CI - High.21,Quartile.34
0,Alabama,AL,1001.0,Autauga,9409.294766,8492.111635,10326.477898,2,11806.223285,NaN,...,88.431482,4,82.494851,NaN,82.539683,21082.0,40.8,36.698437,44.901563,3
1,Alabama,AL,1003.0,Baldwin,7467.596664,7025.900408,7909.292920,1,10335.702691,2827.348216,...,86.017804,2,85.567010,68.864800,84.646255,73058.0,40.1,37.614952,42.585048,3
2,Alabama,AL,1005.0,Barbour,8929.474982,7633.929691,10225.020273,1,9141.333085,NaN,...,85.796041,1,80.341227,NaN,85.656228,7271.0,35.1,29.740946,40.459054,2
3,Alabama,AL,1007.0,Bibb,11741.938889,10057.801626,13426.076153,4,11297.166352,NaN,...,89.905820,2,NaN,NaN,NaN,7044.0,48.8,40.894758,56.705242,4
4,Alabama,AL,1009.0,Blount,9359.122343,8463.490290,10254.754396,1,NaN,5133.144570,...,87.622499,3,NaN,79.539642,81.580997,18384.0,60.2,55.818947,64.581053,4


In [6]:
#clean up data to show only stats in comparision
Unemployment_Percent=stats_df["Unemployment Percent"]
Obesity_Percent=stats_df["Obesity Percent"]
Some_College=stats_df["Some College Percent"]
High_School=stats_df["High School Grad Percent"]
State_Name=stats_df["State"]
County=stats_df["County"]
state_abbr=stats_df["State Abbr"]

#create new Dataframe with just information needed
formatted_df= pd.DataFrame({"State": State_Name,
                        "State Abbr": state_abbr,
                        "County": County,
                        "Unemployment (%)": Unemployment_Percent,
                        "Obesity (%)": Obesity_Percent,
                        "Some College (%)": Some_College,
                        "High School Graduate (%)": High_School
                       })

formatted_df.head()

,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%)
0,Alabama,AL,Autauga,5.271161,36.4,61.942019,90.000000
1,Alabama,AL,Baldwin,5.415263,29.3,63.757100,85.000000
2,Alabama,AL,Barbour,8.648518,44.2,39.723661,86.173469
3,Alabama,AL,Bibb,6.637117,38.4,49.821371,85.000000
4,Alabama,AL,Blount,5.520897,35.8,53.866457,91.206250


In [7]:
formatted_df.count()

State                       3150
State Abbr                  3150
County                      3141
Unemployment (%)            3140
Obesity (%)                 3141
Some College (%)            3141
High School Graduate (%)    2671
dtype: int64

In [8]:
#drop rows missing data
formatted_df.dropna()

,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%)
0,Alabama,AL,Autauga,5.271161,36.4,61.942019,90.000000
1,Alabama,AL,Baldwin,5.415263,29.3,63.757100,85.000000
2,Alabama,AL,Barbour,8.648518,44.2,39.723661,86.173469
3,Alabama,AL,Bibb,6.637117,38.4,49.821371,85.000000
4,Alabama,AL,Blount,5.520897,35.8,53.866457,91.206250
5,Alabama,AL,Bullock,7.171066,40.2,33.716075,87.500000
6,Alabama,AL,Butler,7.128778,36.0,42.221767,83.000000
7,Alabama,AL,Calhoun,6.697384,36.1,56.743524,92.163374
8,Alabama,AL,Chambers,5.543809,38.0,46.389824,85.000000
9,Alabama,AL,Cherokee,5.213652,35.5,55.565627,88.000000


In [9]:
formatted_df.count()

State                       3150
State Abbr                  3150
County                      3141
Unemployment (%)            3140
Obesity (%)                 3141
Some College (%)            3141
High School Graduate (%)    2671
dtype: int64

In [12]:
#Get sample of 25% of population
sample = formatted_df.sample(frac =.30)
sample

,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%)
302,Colorado,CO,San Miguel,3.309969,18.3,75.937122,NaN
2021,North Dakota,ND,McLean,3.608247,35.6,68.361303,NaN
1137,Louisiana,LA,Grant,6.993675,35.9,38.181818,85.000000
829,Iowa,IA,Grundy,3.500670,31.6,77.338651,NaN
1841,New York,NY,Clinton,5.348759,30.1,56.462620,85.066631
804,Iowa,IA,Calhoun,3.873395,33.5,67.971014,96.557018
3026,West Virginia,WV,Mason,7.067067,38.3,50.867594,89.000000
2504,Tennessee,TN,Polk,5.778491,34.4,41.151499,92.500000
2606,Texas,TX,Floyd,5.190058,28.1,48.179272,92.200000
1860,New York,NY,Montgomery,5.672811,29.1,55.195416,77.449909
